In [ ]:
import numpy as np
import pandas as pd

In [ ]:
!pip3 install pyvi

In [ ]:
data = pd.read_csv('./reviews.csv')
data_sample = data.sample(frac = 1)

In [ ]:
reviews = data_sample['Review']
labels = data_sample['Label'].fillna(0.)
reviews.head(10)
reviews.shape[0]

5914

In [69]:
englishwords = pd.read_json("./englishwords.json")
englishwords.set_index("english", inplace = True)
shortform = pd.read_json("./shortform.json")
shortform.set_index("short", inplace = True)
englishwords

,vietnamese
english,
fun,vui
phone,điện thoại
funny,hay ho
concept,ý tưởng
idea,ý tưởng
smartphone,điện thoại thông minh
not,không
yes,có
capable,có thể


In [70]:
shortform

,long
short,
hnay,hôm nay
k,không
ko,không
hqua,hôm qua
htrc,hôm trước
bh,bây giờ
bg,bây giờ
h,giờ
cn,công nghệ


In [ ]:
reviews.loc[1300]

' Bạn bè bảo đi thử quán gia truyền nên đi Thề luôn chưa thấy quán bít tết nào tệ như điểm cũng dễ nhưng cầu thang di chuyển thì bé kiểu cổ ngày xưa bậc cao mà người đi phải lách Mình ngồi tầng nhưng nhìn chung chật nóng bàn kê san sát buổi tối mà đèn thì bật như kiểu tiết cảm tưởng như thời bao viên phục vụ Bàn người mà mang có đĩa dưa chuột bé gọi để hỏi thêm dưa chuột thì gọi mãi không thấy suất ăn của nó mới là quá kinh Mình gọi bánh mỳ chảo dăm bông trứng lòng đào thì mang cái chảo nhìn như đồ ăn trứng đến lòng trắng còn chảy nhão ra chưa thịt thì khô và Suất bít tết thì ngấy kinh một lượng mỡ ngập ngụa mà không có trứng cháy ở khoai tây lèo pate hộp nên cứng mà thịt bò thì dai thôi rồi cắt còn chẳng cắt hiểu vì sao quán vẫn khẩu vị tệ Thề không bao giờ quay trở'

In [78]:
# Loại bỏ kí tự đặc biệt và chuyển thành chữ cái viết thường
import re 

def preprocessor(text):
    """ Return a cleaned version of text
    """
    # Remove HTML markup
    text = re.sub('<[^>]*>', '', text)
    # Save emoticons for later appending
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ' '))
    
    #Loại bỏ các chữ cái lặp nhau
    length = len(text)
    char = 0
    while char <length-1:
        if text[char] == text[char+1]:
            text = text[:char]+text[char+1:]
            #print(text)
            length-=1
            continue
        char+=1
    #loại bỏ số  và các chữ cái lạ:
    digits_and_characters = 'aăâbcdđeêfghijklmnoôơpqrstuưvxywzáàảãạắằẳẵặấầẩẫậéèẻẽẹếềểễệíìỉĩịóòỏõọốồổỗộớờởỡợúùủũụứừửữựýỳỷỹỵ(?::|;|=)(?:-)?(?:\)|\(|D|P) '
    text = ''.join([i for i in text if i in digits_and_characters])

    text_split = text.split()
    #print(text_split)
    for i in range(len(text_split)):
      t = text_split[i]
      if t in englishwords.index:
        text_split[i] = str(englishwords.loc[t, "vietnamese"])
      elif t in shortform.index:
        text_split[i] = str(shortform.loc[t,"long"]) 
    #print(text_split)
    text = ' '.join(text_split)
    return text


In [76]:
print(preprocessor('ngoonnnnnnnn quáaaaaaa trời ơiiii oiiii not yes ko <abcxyz> <345345345> |||| 12345맛있고'))

['ngon', 'quáa', 'trời', 'ơi', 'oi', 'not', 'yes', 'ko']
['ngon', 'quáa', 'trời', 'ơi', 'oi', 'không', 'có', 'không']
ngon quáa trời ơi oi không có không


In [79]:
# get rid of special characters and convert to lower case
for i in range (reviews.shape[0]):
    reviews.loc[i] = preprocessor(reviews.loc[i])
reviews.head(10)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


4966    thậpkỷ tôi đến quán rất thường saunày thithoản...
1688    vì trường mình ngày trước gần đây nên có qua ă...
2732    hômnay sinhnhật em e mình thích ăn mà các quán...
4253    mình đi ờ thì vào cái là các em mang menu ra đ...
3715    ngọc mai vàng ưng nhất là cái khôngian và vừa ...
798                          salad cáhồi tươi ngon hấpdẫn
2853      ng chỉ nên ăn cái dễ ngán chấtlượng kém hơn trc
720     chỗ bufet dườngnhư là quenthuộc nhất hànội khô...
4302              quán tuyệt ngon có congái xinhxắn hihih
4107    bị nghiện đi ăn bufet ở đây thích món bò nướng...
Name: Review, dtype: object

In [80]:
reviews.loc[1300]

'bạnbè bảo đi thử quán giatruyền nên đi thề luôn chưa thấy quán bít tết nào tệ như điểm cũng dễ nhưng cầuthang dichuyển thì bé kiểu cổ ngàyxưa bậc cao mà người đi phải lách mình ngồi tầng nhưng nhìnchung chật nóng bàn kê sansát buổi tối mà đèn thì bật như kiểu tiết cảmtưởng như thời baoviên phụcvụ bàn người mà mang có đĩa dưachuột bé gọi để hỏi thêm dưachuột thì gọi mãi không thấy suất ăn của nó mới là quá kinh mình gọi bánh mỳ chảo dăm bông trứng lòngđào thì mang cái chảo nhìn như đồ ăn trứng đến lòngtrắng còn chảy nhão ra chưa thịt thì khô và suất bít tết thì ngấy kinh một lượng mỡ ngậpngụa mà không có trứng cháy ở khoaitây lèo pate hộp nên cứng mà thịt bò thì dai thôi rồi cắt còn chẳng cắt hiểu vì sao quán vẫn khẩuvị tệ thề không baogiờ quay trở'

In [81]:
from pyvi import ViTokenizer
for i in range (reviews.shape[0]):
    reviews.loc[i] = ViTokenizer.tokenize(reviews.loc[i]).replace('_','')
    

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [82]:
print(reviews.loc[1300], labels.loc[1300])

bạnbè bảo đi thử quán giatruyền nên đi thề luôn chưa thấy quán bít tết nào tệ như điểm cũng dễ nhưng cầuthang dichuyển thì bé kiểu cổ ngàyxưa bậc cao mà người đi phải lách mình ngồi tầng nhưng nhìnchung chật nóng bàn kê sansát buổi tối mà đèn thì bật như kiểu tiết cảmtưởng như thời baoviên phụcvụ bàn người mà mang có đĩa dưachuột bé gọi để hỏi thêm dưachuột thì gọi mãi không thấy suất ăn của nó mới là quá kinh mình gọi bánh mỳ chảo dăm bông trứng lòngđào thì mang cái chảo nhìn như đồ ăn trứng đến lòngtrắng còn chảy nhão ra chưa thịt thì khô và suất bít tết thì ngấy kinh một lượng mỡ ngậpngụa mà không có trứng cháy ở khoaitây lèo pate hộp nên cứng mà thịt bò thì dai thôi rồi cắt còn chẳng cắt hiểu vì sao quán vẫn khẩuvị tệ thề không baogiờ quay trở -1.0


In [83]:
from collections import Counter

vocab = Counter()
for twit in reviews:
    for word in twit.split(' '):
        vocab[word] += 1

vocab.most_common(10)

[('không', 9793),
 ('mình', 8883),
 ('ăn', 8704),
 ('là', 7181),
 ('thì', 7050),
 ('có', 6844),
 ('và', 5857),
 ('cũng', 4739),
 ('quán', 4704),
 ('rất', 3730)]

In [84]:
with open('./stopwords_vn.txt') as f:
    stopwords_vn = f.read().splitlines()
stopwords_vn[:10]

['alô', 'aha', 'ai', 'aiai', 'ainấy', 'aiđó', 'alô', 'amen', 'anh', 'anhấy']

In [85]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [86]:
from nltk.corpus import stopwords
stopwords_eng = stopwords.words('english')

In [87]:
stopwords = stopwords_vn + stopwords_eng

In [88]:
vocab_reduced = Counter()
# Go through all of the items of vocab using vocab.items() and pick only words that are not in 'stop_words' 
# and save them in vocab_reduced
for w, c in vocab.items():
    if not w in stopwords:
        vocab_reduced[w]=c

vocab_reduced.most_common(10)

[('không', 9793),
 ('quán', 4704),
 ('mà', 3632),
 ('lại', 3094),
 ('đồ', 3053),
 ('ngon', 2912),
 ('đi', 2855),
 ('gọi', 2564),
 ('nhânviên', 2363),
 ('món', 2172)]

In [140]:
from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer(stop_words = stopwords, min_df = 5, ngram_range= (2,3))
bag = bow.fit_transform(reviews)
print(bag.shape)
print(bow.get_feature_names())

(5914, 5413)
['aeon mal', 'anhchị chủ', 'anhchị nhânviên', 'anhchị phụcvụ', 'bachỉ bò', 'banđầu quán', 'bao không', 'bef steak', 'bingsu ngon', 'bingsu trà', 'bit tết', 'biển quán', 'bufet hảisản', 'bufet không', 'bufet lẩu', 'bufet lẩu nướng', 'bufet mà', 'bufet món', 'bufet nướng', 'bufet thịt', 'buồn không', 'bàitrí khá', 'bàn bàn', 'bàn bé', 'bàn bìnhthường', 'bàn bảo', 'bàn bếp', 'bàn bọn', 'bàn chỗ', 'bàn chờ', 'bàn cạnh', 'bàn cạnh gọi', 'bàn cửasổ', 'bàn fody', 'bàn ghế', 'bàn ghế không', 'bàn gọi', 'bàn hai', 'bàn hôm', 'bàn hơi', 'bàn khá', 'bàn không', 'bàn lại', 'bàn mà', 'bàn món', 'bàn ngoàitrời', 'bàn nhânviên', 'bàn nướng', 'bàn order', 'bàn phụcvụ', 'bàn quán', 'bàn sát', 'bàn table', 'bàn tablenow', 'bàn trc', 'bàn tròn', 'bàn trống', 'bàn tầng', 'bàn view', 'bàn xong', 'bàn xungquanh', 'bàn đi', 'bàn đông', 'bàn đấy', 'bàn đồ', 'bàn độ', 'bàn đợi', 'bày trí', 'bánh bánh', 'bánh bánh bột', 'bánh bánh ngon', 'bánh bé', 'bánh bình', 'bánh bún', 'bánh bảo', 'bánh bột', '

In [141]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words = stopwords, min_df = 5, ngram_range= (2,3))
# Feed the tf-idf Vectorizer with twits using fit_transform()
tfidf_vec = tfidf.fit_transform(reviews)

# Formatting the number to 2 digits after the decimal point by showing on this notebook
np.set_printoptions(precision=2)
# To print array in one line
np.set_printoptions(linewidth=np.inf)
print(tfidf.get_feature_names())
print(tfidf_vec.shape)

['aeon mal', 'anhchị chủ', 'anhchị nhânviên', 'anhchị phụcvụ', 'bachỉ bò', 'banđầu quán', 'bao không', 'bef steak', 'bingsu ngon', 'bingsu trà', 'bit tết', 'biển quán', 'bufet hảisản', 'bufet không', 'bufet lẩu', 'bufet lẩu nướng', 'bufet mà', 'bufet món', 'bufet nướng', 'bufet thịt', 'buồn không', 'bàitrí khá', 'bàn bàn', 'bàn bé', 'bàn bìnhthường', 'bàn bảo', 'bàn bếp', 'bàn bọn', 'bàn chỗ', 'bàn chờ', 'bàn cạnh', 'bàn cạnh gọi', 'bàn cửasổ', 'bàn fody', 'bàn ghế', 'bàn ghế không', 'bàn gọi', 'bàn hai', 'bàn hôm', 'bàn hơi', 'bàn khá', 'bàn không', 'bàn lại', 'bàn mà', 'bàn món', 'bàn ngoàitrời', 'bàn nhânviên', 'bàn nướng', 'bàn order', 'bàn phụcvụ', 'bàn quán', 'bàn sát', 'bàn table', 'bàn tablenow', 'bàn trc', 'bàn tròn', 'bàn trống', 'bàn tầng', 'bàn view', 'bàn xong', 'bàn xungquanh', 'bàn đi', 'bàn đông', 'bàn đấy', 'bàn đồ', 'bàn độ', 'bàn đợi', 'bày trí', 'bánh bánh', 'bánh bánh bột', 'bánh bánh ngon', 'bánh bé', 'bánh bình', 'bánh bún', 'bánh bảo', 'bánh bột', 'bánh bột lọc'

In [142]:
tfidf_vec.shape

(5914, 5413)

In [143]:
bag.shape

(5914, 5413)

In [132]:
spliter = int(bag.shape[0]*0.8)
bag_train = bag[:spliter, :]
bag_test = bag[spliter:, :]
tfidf_train = tfidf_vec[:spliter,:]
tfidf_test = tfidf_vec[spliter:, :]
labels_train = labels[:spliter]
labels_test = labels[spliter:]

In [133]:
from sklearn.naive_bayes import MultinomialNB
nbc = MultinomialNB()

In [124]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

In [114]:
print(labels_train.isna().sum())

0


In [144]:
nbc.fit(bag_train.toarray(), labels_train)
labels_pred = nbc.predict(bag_test.toarray())

In [145]:
print(accuracy_score(labels_test, labels_pred))
print(f1_score(labels_test, labels_pred, average = 'macro'))

0.7743026204564666
0.6333905747717835


In [146]:
nbc.fit(tfidf_train.toarray(), labels_train)
labels_pred = nbc.predict(tfidf_test.toarray())

In [147]:
print(accuracy_score(labels_test, labels_pred))
print(f1_score(labels_test, labels_pred, average = 'macro'))

0.7861369399830939
0.5576277608256656


In [119]:
from sklearn.linear_model import LogisticRegression


In [148]:
lor = LogisticRegression()
lor.fit(bag_train.toarray(), labels_train)
labels_pred = lor.predict(bag_test.toarray())
print(accuracy_score(labels_test, labels_pred))
print(f1_score(labels_test, labels_pred, average = 'macro'))

0.760777683854607
0.6130800830089501


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [149]:
lor.fit(tfidf_train.toarray(), labels_train)
labels_pred = lor.predict(tfidf_test.toarray())
print(accuracy_score(labels_test, labels_pred))
print(f1_score(labels_test, labels_pred, average = 'macro'))
print(classification_report(labels_test, labels_pred))

0.7996618765849535
0.592010655009258
              precision    recall  f1-score   support

        -1.0       0.78      0.88      0.83       495
         0.0       0.55      0.04      0.08       144
         1.0       0.82      0.93      0.87       544

    accuracy                           0.80      1183
   macro avg       0.72      0.62      0.59      1183
weighted avg       0.77      0.80      0.76      1183



In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(max_depth = None)

In [ ]:
dtc.fit(bag_train.toarray(), labels_train)
labels_pred = dtc.predict(bag_test.toarray())
print(accuracy_score(labels_test, labels_pred))
print(f1_score(labels_test, labels_pred, average = 'macro'))

0.5908706677937448
0.47942422776472665


In [ ]:
dtc.fit(tfidf_train.toarray(), labels_train)
labels_pred = dtc.predict(tfidf_test.toarray())
print(accuracy_score(labels_test, labels_pred))
print(f1_score(labels_test, labels_pred, average = 'macro'))

0.6052409129332206
0.48527338673250947


In [106]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(max_depth = None, random_state = 0)

In [122]:
rfc.fit(bag_train.toarray(), labels_train)
labels_pred = rfc.predict(bag_test.toarray())
print(accuracy_score(labels_test, labels_pred))
print(f1_score(labels_test, labels_pred, average = 'macro'))

0.7692307692307693
0.5779230246118193


In [123]:
rfc.fit(tfidf_train.toarray(), labels_train)
labels_pred = rfc.predict(tfidf_test.toarray())
print(accuracy_score(labels_test, labels_pred))
print(f1_score(labels_test, labels_pred, average = 'macro'))

0.7658495350803043
0.5641697104385265


In [151]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier()

In [152]:
mlp.fit(bag_train, labels_train)
labels_pred_mlp = mlp.predict(bag_test)
print(classification_report(labels_test, labels_pred_mlp))

              precision    recall  f1-score   support

        -1.0       0.78      0.79      0.79       495
         0.0       0.26      0.22      0.24       144
         1.0       0.81      0.83      0.82       544

    accuracy                           0.74      1183
   macro avg       0.62      0.62      0.62      1183
weighted avg       0.73      0.74      0.74      1183



In [155]:
mlp.fit(tfidf_train, labels_train)
labels_pred_mlp = mlp.predict(tfidf_test)
print(classification_report(labels_test, labels_pred_mlp))

              precision    recall  f1-score   support

        -1.0       0.77      0.80      0.78       495
         0.0       0.31      0.18      0.23       144
         1.0       0.81      0.87      0.84       544

    accuracy                           0.76      1183
   macro avg       0.63      0.62      0.62      1183
weighted avg       0.73      0.76      0.74      1183



In [153]:
import lightgbm as lgbm

In [157]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean= False)

In [158]:
bag_train = sc.fit_transform(bag_train)
bag_test = sc.fit_transform(bag_test)

In [168]:
y_train = np.array(labels_train)+1
y_train
y_test = np.array(labels_test)+1

In [188]:
d_train = lgbm.Dataset(bag_train, label=y_train)
params = {}
params['learning_rate'] = 0.003
params['boosting_type'] = 'gbdt'
params['objective'] = 'binary'
params['metric'] = 'binary_logloss'
params['num_leaves'] = 10
clf_lgb = lgbm.train(params, d_train, 100)

In [214]:
y_train_pred = clf_lgb.predict(bag_train)
y_pred = clf_lgb.predict(bag_test)
print(y_pred)
y_classified = []
for i in range(len(y_pred)):
  if y_pred[i] > 0.62:
    y_classified.append(2)
  elif y_pred[i] < 0.52:
    y_classified.append(0)
  else:
    y_classified.append(1)
print(y_classified)
print(classification_report(y_test, y_classified)) 

[0.64 0.57 0.48 ... 0.66 0.48 0.66]
[2, 1, 0, 2, 1, 1, 2, 2, 0, 2, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 0, 1, 1, 0, 2, 1, 0, 1, 1, 2, 1, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 0, 2, 2, 0, 0, 1, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 1, 2, 1, 2, 1, 1, 1, 2, 1, 2, 2, 1, 0, 1, 0, 1, 0, 2, 2, 2, 0, 0, 2, 0, 1, 2, 2, 0, 2, 0, 2, 2, 0, 2, 2, 1, 2, 2, 1, 1, 2, 0, 1, 2, 2, 1, 2, 1, 2, 0, 1, 2, 0, 1, 2, 2, 2, 0, 2, 0, 1, 1, 2, 2, 0, 0, 2, 2, 1, 2, 2, 0, 1, 1, 2, 0, 1, 0, 2, 1, 0, 1, 1, 1, 2, 1, 2, 2, 1, 1, 2, 0, 2, 1, 1, 2, 2, 2, 0, 2, 2, 1, 2, 2, 2, 0, 2, 2, 0, 2, 0, 2, 1, 0, 2, 2, 2, 1, 0, 2, 2, 0, 1, 2, 1, 2, 1, 2, 0, 2, 1, 2, 0, 2, 1, 0, 2, 2, 1, 2, 1, 2, 0, 2, 1, 1, 2, 0, 1, 2, 1, 2, 0, 1, 1, 2, 1, 1, 1, 2, 2, 2, 0, 2, 2, 2, 1, 2, 2, 2, 2, 0, 0, 0, 2, 2, 2, 1, 1, 1, 2, 2, 2, 2, 2, 0, 2, 0, 1, 1, 1, 2, 1, 2, 2, 2, 2, 1, 2, 1, 2, 0, 2, 0, 0, 1, 1, 1, 0, 2, 1, 2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1, 2, 1, 2, 2, 2, 2, 1, 2, 1, 1, 2, 2, 0, 1, 0, 2, 2, 2, 2, 0, 2, 2, 0, 2, 0, 2, 0, 2, 0, 1, 1, 1, 